# MODEL 5 - MobileNetV3 with feature vector euclidian distance

- Use hard-swish
- Loss: BinaryCrossentropy


In [27]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.python.data import AUTOTUNE
from tensorflow.keras import layers as kl
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks as cb
import tensorflow_addons as tfa

from src.math.euclidian_distance import euclidian_distance
from src.nn.activations.hard_swish import hard_swish
from src.nn.layers.image_resize_fit import ResizeImageToFit
from src.nn.losses.contrastive_loss import contrastive_loss
from src.nn.layers.euclidian_distance import EuclidianDistance


if os.getcwd().startswith("/Users"): os.chdir(os.getcwd().split("/notebooks", 1)[0])

from src.config.notebook_config import NotebookConfig

from src.evaluation.evaluation import Evaluation

from src.data.utility.tf_dataset_transformer import TfDatasetTransformer


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config

In [30]:
MODEL_NAME = "model_5"

import src.data.dataset.dataset_v3_loader as dataset_loader # V3
DATASET_LOADER = dataset_loader

config = NotebookConfig(MODEL_NAME, dataset_loader.dataset_name())

BATCH_SIZE = 128

print(config.summary())

Model: model_5
Output: ./output/model_5_v3
Output for Tensorboard: ./output/model_5_v3
Saved model path: ./models/model_5_v3

Saved model path: ./models/model_5_v3



# Data Loading

In [22]:
train_dataset = DATASET_LOADER.load_train_v3()
val_dataset = DATASET_LOADER.load_validation_v3()

In [23]:
pair_tuples = list(map(lambda pair: (pair.image_a, pair.image_b, pair.similar), train_dataset.image_pairs))
pair_array = np.array(pair_tuples)

tf_dataset_transformer =  TfDatasetTransformer(image_shape=config.image_shape, rescale=False)

train_tf_dataset = tf_dataset_transformer.transform_to_tf_dataset(train_dataset, shuffle=True, batch_size=BATCH_SIZE, cache_path="./tf_cache/train_dataset")
val_tf_dataset = tf_dataset_transformer.transform_to_tf_dataset(val_dataset, shuffle=False, batch_size=128, cache_path="./tf_cache/val_dataset")

# Model

In [26]:
def create_common_model() -> keras.Model:
    mobnet = keras.applications.MobileNetV3Large(input_shape=config.image_shape.to_shape(), include_top=False, pooling="avg", dropout_rate=0.2)
    mobnet.trainable = False

    input = keras.Input(shape=(None, None, 3))
    x = ResizeImageToFit(height=config.image_shape.height, width=config.image_shape.width)(input)
    x = mobnet(x, training=False)
    x = kl.Dense(1280, activation=hard_swish)(x)
    return keras.Model(inputs=input, outputs=x)


def create_model() -> keras.Model:
    input_a = keras.Input(shape=(None, None, 3))
    input_b = keras.Input(shape=(None, None, 3))

    common_model = create_common_model()

    print(common_model.summary())

    layer_a = common_model(input_a)
    layer_b = common_model(input_b)

    merge_layer = EuclidianDistance()([layer_a, layer_b])
    merge_layer = kl.Dense(1, activation="sigmoid")(merge_layer)

    return keras.Model(inputs=[input_a, input_b], outputs=merge_layer, name=MODEL_NAME)


lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.005, decay_steps=999, decay_rate=0.8)

model = create_model()
model.compile(
    optimizer=optimizers.Adam(learning_rate=lr_schedule), loss=contrastive_loss, metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resize_image_to_fit (Resize  (None, 224, 224, 3)      0         
 ImageToFit)                                                     
                                                                 
 MobilenetV3large (Functiona  (None, 960)              2996352   
 l)                                                              
                                                                 
 dense (Dense)               (None, 1280)              1230080   
                                                                 
Total params: 4,226,432
Trainable params: 1,230,080
Non-trainable params: 2,996,352
_________________________________________________________________
None


NameError: name 'EuclidianDistance' is not defined

In [ ]:
keras.utils.plot_model(model, show_shapes=True, expand_nested=False)

In [ ]:
save_cb = cb.ModelCheckpoint(config.model_output_path, save_best_only=True)
tensorboard_cb = cb.TensorBoard(log_dir=config.tensorboard_log_dir)

history = model.fit(train_tf_dataset, epochs=40, workers=8, max_queue_size=20, validation_data=val_tf_dataset, callbacks=[save_cb, tensorboard_cb])

pd.DataFrame(history.history).plot()

In [12]:
import gc
gc.collect(2)#%%

559

# Evaluation

In [7]:
a = 6
a

6

In [6]:
model: keras.Model = keras.models.load_model(config.saved_model_path)

In [18]:
evaluation = Evaluation(dataset=val_dataset, image_shape=config.image_shape, model=model)
iter = evaluation.tf_dataset.as_numpy_iterator()

In [19]:

for i in evaluation.tf_dataset:
    pass

In [96]:
evaluation.print_f1_report(threshold=0.5)
evaluation.print_separate_f1_reports(threshold=0.5)
evaluation.plot_roc()

In [ ]:
evaluation.show_images(range(8000, 8100))

In [85]:
history = model.fit(train_tf_dataset, epochs=1, workers=8, max_queue_size=20, validation_data=val_tf_dataset, callbacks=[save_cb, tensorboard_cb], steps_per_epoch=1)


1/1 [==============================] - ETA: 0s - loss: 2.0552e-09 - accuracy: 1.0000 - f1-micro: 0.0000e+00

KeyboardInterrupt: 

In [75]:
pair = val_dataset.image_pairs[5]
pred = model.predict([np.expand_dims(pair.image_a_array(), axis=0), np.expand_dims(pair.image_b_array(), axis=0)])
pred

array([[0.00631198]], dtype=float32)

In [76]:
model.weights

[<tf.Variable 'Conv/kernel:0' shape=(3, 3, 3, 16) dtype=float32, numpy=
 array([[[[ 4.74257290e-01,  3.84327859e-01, -5.35630465e-01,
            3.55508886e-02,  2.88742006e-01,  6.94768667e-01,
            8.48755240e-02, -4.37884361e-01,  3.74123193e-02,
            2.09846795e-01,  7.10104853e-02,  2.88548261e-01,
            4.00461972e-01, -5.21113813e-01, -2.53070295e-01,
           -3.63211691e-01],
          [-2.07101926e-01, -7.34134853e-01, -1.04572845e+00,
           -2.03642268e-02,  4.34004098e-01, -4.76945043e-01,
           -4.75457720e-02, -8.79060984e-01, -6.75275102e-02,
            2.86088586e-01,  1.08445607e-01,  6.58086061e-01,
            2.90191799e-01, -9.57054377e-01,  5.52441657e-01,
           -4.56653982e-01],
          [-2.07804918e-01,  3.55969936e-01, -2.68229634e-01,
           -1.49971228e-02,  2.06070811e-01, -2.55605727e-01,
            9.15222242e-02, -2.33704612e-01,  6.88022422e-03,
            1.35320961e-01, -6.68843910e-02,  1.93160713e-01,
  

In [84]:
loss_fn = losses.binary_crossentropy

# Iterate over the batches of the dataset.
for step, (x, y) in enumerate(val_tf_dataset):

    # Open a GradientTape.
    with tf.GradientTape() as tape:

        # Forward pass.
        logits = model(x)

        # Loss value for this batch.
        loss = loss_fn(y, tf.squeeze(logits))

    # Get gradients of the loss wrt the weights.
    gradients = tape.gradient(loss, model.trainable_weights)

    # Update the weights of our linear layer.
    # .apply_gradients(zip(gradients, linear_layer.trainable_weights))

    # Logging.
    if step % 100 == 0:
        print("Step:", step, "Loss:", float(loss))

KeyboardInterrupt: 